# Retrieval Augmented FuctionCallingAgent


In this tutorial, we demonstrate how to utilize our `FunctionCallingAgent `implementation alongside a tool retriever.

The goal is to construct an agent that leverages the function-calling API of LLMs (such as OpenAI, Anthropic, or Mistral) to manage and index an arbitrary number of tools.

Our indexing and retrieval modules are designed to simplify the process, addressing the challenge of accommodating numerous functions within a prompt.

`FunctionCallingAgent` uses LLM function calling capabilities to complete the task.

In [ ]:
!pip install llama-index
!pip install llama-index-llms-anthropic
!pip install llama-index-llms-mistralai

## Simple Calculator Tools

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.mistralai import MistralAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

### Define Tools

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers with and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers with and returns the result integer"""
    return a - b

def useless(a: int, b: int) -> int:
    """Toy useless function."""
    pass

In [ ]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

useless_tools = [
    FunctionTool.from_defaults(fn=useless, name=f"useless_{str(idx)}")
    for idx in range(50)
]

In [ ]:
all_tools = [multiply_tool] + [add_tool] + [subtract_tool] + useless_tools

### Setup API Key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR OPENAI API KEY'

### Create ObjectIndex

In [ ]:
# define an "object" index over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

####  OpenAI GPT-4

In [ ]:
llm = OpenAI(model="gpt-4")

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=2),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("What's 500 multiplied by 10?")

Added user message to memory: What's 500 multiplied by 10?
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== LLM Response ===
500 multiplied by 10 is 5000.


In [ ]:
print(response)

assistant: 500 multiplied by 10 is 5000.


In [ ]:
response = agent.chat("What's 500 multiplied by 10 and then subtract 1000?")

Added user message to memory: What's 500 multiplied by 10 and then subtract 1000?
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== Calling Function ===
Calling function: subtract with args: {"a": 5000, "b": 1000}
=== Function Output ===
4000
=== LLM Response ===
500 multiplied by 10 and then subtracting 1000 gives 4000.


In [ ]:
print(response)

assistant: 500 multiplied by 10 and then subtracting 1000 gives 4000.


#### Anthropic Sonnet

In [ ]:
os.environ['ANTHROPIC_API_KEY'] = 'YOUR ANTHROPIC API KEY'

In [ ]:
llm = Anthropic(model="claude-3-sonnet-20240229")
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=2),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("What's 500 multiplied by 10?")

Added user message to memory: What's 500 multiplied by 10?
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== LLM Response ===
So 500 multiplied by 10 is 5000.


In [ ]:
print(response)

assistant: So 500 multiplied by 10 is 5000.


In [ ]:
response = agent.chat("What's 500 multiplied by 10 and then subtract 1000?")

Added user message to memory: What's 500 multiplied by 10 and then subtract 1000?
=== LLM Response ===
Okay, let's break this down into steps:
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== LLM Response ===
First, 500 multiplied by 10 is 5000.
=== Calling Function ===
Calling function: subtract with args: {"a": 5000, "b": 1000}
=== Function Output ===
4000
=== LLM Response ===
Then, subtracting 1000 from 5000 gives 4000.

Therefore, the result of 500 multiplied by 10 and then subtracting 1000 is 4000.


In [ ]:
print(response)

assistant: Then, subtracting 1000 from 5000 gives 4000.

Therefore, the result of 500 multiplied by 10 and then subtracting 1000 is 4000.


#### MistralAI

In [ ]:
os.environ['MISTRAL_API_KEY'] = 'YOUR MISTRALAI API KEY'

In [ ]:
llm = MistralAI(model="mistral-large-latest")
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=2),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("What's 500 multiplied by 10?")

Added user message to memory: What's 500 multiplied by 10?
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== LLM Response ===
The result of multiplying 500 by 10 is 5000.


In [ ]:
print(response)

assistant: The result of multiplying 500 by 10 is 5000.


In [ ]:
response = agent.chat("What's 500 multiplied by 10 and then subtract 1000?")

Added user message to memory: What's 500 multiplied by 10 and then subtract 1000?
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== Calling Function ===
Calling function: subtract with args: {"a": 5000, "b": 1000}
=== Function Output ===
4000
=== LLM Response ===
The result of multiplying 500 by 10 and then subtracting 1000 is 4000.


In [ ]:
print(response)

assistant: The result of multiplying 500 by 10 and then subtracting 1000 is 4000.


## RAG QueryEngine Tools

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-04-12 22:32:15--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.04s   

2024-04-12 22:32:16 (40.5 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-04-12 22:32:16--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

### Load Data And Create Index

We will by default use OpenAI Embeddings for building index.

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

### Create QueryEngines

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

### Define Tools

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
obj_index = ObjectIndex.from_objects(
    query_engine_tools,
    index_cls=VectorStoreIndex,
)

#### OpenAI GPT-4

In [ ]:
llm = OpenAI(model="gpt-4")
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=1),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat(
    "What is the revenue of Uber in 2021?"
)

Added user message to memory: What is the revenue of Uber in 2021?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What is the revenue of Uber in 2021?"}
=== Function Output ===
$17,455
=== LLM Response ===
The revenue of Uber in 2021 was $17,455 million.


In [ ]:
print(str(response))

assistant: The revenue of Uber in 2021 was $17,455 million.


In [ ]:
response = agent.chat(
    "What is the revenue of Lyft in 2021?"
)

Added user message to memory: What is the revenue of Lyft in 2021?
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What is the revenue of Lyft in 2021?"}
=== Function Output ===
$3,208,323
=== LLM Response ===
The revenue of Lyft in 2021 was $3,208,323,000.


In [ ]:
print(str(response))

assistant: The revenue of Lyft in 2021 was $3,208,323,000.


#### Anthropic Sonnet

In [ ]:
llm = Anthropic(model="claude-3-sonnet-20240229")
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=1),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat(
    "What is the revenue of Uber in 2021?"
)

Added user message to memory: What is the revenue of Uber in 2021?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was Uber's revenue in 2021?"}
=== Function Output ===
Uber's revenue in 2021 was $17,455 million.
=== LLM Response ===
So Uber's revenue in 2021 was $17,455 million.


In [ ]:
print(response)

assistant: So Uber's revenue in 2021 was $17,455 million.


In [ ]:
response = agent.chat(
    "What is the revenue of Lyft in 2021?"
)

Added user message to memory: What is the revenue of Lyft in 2021?
=== LLM Response ===
Okay, let me check Lyft's 2021 revenue using the provided tool:
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What was Lyft's revenue in 2021?"}
=== Function Output ===
Lyft's revenue in 2021 was $3,208,323.
=== LLM Response ===
So according to Lyft's 10-K filing, their revenue in 2021 was $3,208,323.


In [ ]:
print(response)

assistant: So according to Lyft's 10-K filing, their revenue in 2021 was $3,208,323.


### MistralAI

In [ ]:
llm = MistralAI(model="mistral-large-latest")
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=1),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat(
    "What is the revenue of Uber in 2021?"
)

Added user message to memory: What is the revenue of Uber in 2021?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What is the revenue of Uber in 2021?"}
=== Function Output ===
The revenue of Uber in 2021 was $17,455 million.
=== LLM Response ===
The revenue of Uber in 2021 was $17,455 million.


In [ ]:
print(response)

assistant: The revenue of Uber in 2021 was $17,455 million.


In [ ]:
response = agent.chat(
    "What is the revenue of Lyft in 2021?"
)

Added user message to memory: What is the revenue of Lyft in 2021?
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What is the revenue of Lyft in 2021?"}
=== Function Output ===
The revenue of Lyft in 2021 was $3,208,323,000.
=== LLM Response ===
The revenue of Lyft in 2021 was $3,208,323,000.


In [ ]:
print(response)

assistant: The revenue of Lyft in 2021 was $3,208,323,000.
